## IMDB Movie Review Sentiment Analysis

In [79]:
from bs4 import BeautifulSoup
import re, string, unicodedata
# from nltk import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import LancasterStemmer, WordNetLemmatizer

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_links_characters(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub('\[[^]]*\]', '', text)
    return text

def regular_preprocess(text):
    text = remove_html(text)
    text = remove_links_characters(text)
    return text

def advanced_preprocess(text):
    return text

def preprocess(data, advanced=False):
    refined_data=[]

    if advanced:
        for data_point in data:
            refined_data.append(advanced_preprocess(data_point))
    else:
        for data_point in data:
            refined_data.append(regular_preprocess(data_point))

    return refined_data


In [85]:
# copy contents of all files in both folders into a list
import glob
import os
from preprocessing import *
from sklearn.model_selection import train_test_split

train_data = []
test_data = []

# train data
train_neg = glob.glob(os.path.join(os.getcwd(), "Dataset/train/neg", "*.txt"))
for f_path in train_neg:
    with open(f_path) as f:
        train_data.append(f.read())

train_pos = glob.glob(os.path.join(os.getcwd(), "Dataset/train/pos", "*.txt"))
for f_path in train_pos:
    with open(f_path) as f:
        train_data.append(f.read())
# print(train_data[0])
# print(preprocess(train_data[0]))

# test data
test_files = glob.glob(os.path.join(os.getcwd(), "Dataset/test", "*.txt"))
for f_path in test_files:
    with open(f_path) as f:
        test_data.append(f.read())

# targets: first 12500 are pos, next 12500 are neg
targets = [1 if i<12500 else 0 for i in range(25000)]

train_data_clean = preprocess(train_data, advanced=False)
test_data_clean = preprocess(test_data, advanced=False)

# splitting the data
X_train, X_validation, y_train, y_validation = train_test_split(train_data, targets, train_size=0.8, test_size=0.2, random_state=1)
# print(train_data_clean[0])
# print(test_data_clean[0])
# print(y_train[0])

### Only CountVectorizer(binary=True)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words vectorization
cv = CountVectorizer(binary=True).fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

In [58]:
from sklearn.naive_bayes import MultinomialNB

clf_NB = MultinomialNB().fit(X_train_counts, y_train)
y_pred = clf_NB.predict(X_validation_counts)

# X_train_normalized = np.array(X_train_normalized)

# print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2488
           1       0.84      0.88      0.86      2512

   micro avg       0.86      0.86      0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000

Accuracy % =  0.8556


In [59]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression().fit(X_train_counts, y_train)
y_pred = clf_LR.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2488
           1       0.89      0.87      0.88      2512

   micro avg       0.88      0.88      0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

Accuracy % =  0.8814


In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# clf_DT = DecisionTreeClassifier().fit(X_train_counts, y_train)
# y_pred = clf_DT.predict(X_validation_counts)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [60]:
from sklearn.svm import LinearSVC

clf_SVM = LinearSVC().fit(X_train_counts, y_train)
y_pred = clf_SVM.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      2488
           1       0.87      0.86      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8656


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### CountVectorizer(binary=True) with tf-idf

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics

# Bag of Words vectorization
cv = CountVectorizer(binary=True).fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

# tfidf
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_validation_tfidf = tfidf_transformer.transform(X_validation_counts)

# normalization
normalizer_tranformer = Normalizer().fit(X=X_train_tfidf)
X_train_normalized = normalizer_tranformer.transform(X_train_tfidf)
X_validation_normalized = normalizer_tranformer.transform(X_validation_tfidf)

In [62]:
clf_NB = MultinomialNB().fit(X_train_normalized, y_train)
y_pred = clf_NB.predict(X_validation_normalized)

# X_train_normalized = np.array(X_train_normalized)

print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

(20000, 68499)
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      2488
           1       0.85      0.89      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8654


In [63]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression().fit(X_train_normalized, y_train)
y_pred = clf_LR.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.88      0.91      0.90      2488
           1       0.91      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8944


In [64]:
# from sklearn.tree import DecisionTreeClassifier

# clf_DT = DecisionTreeClassifier().fit(X_train_normalized, y_train)
# y_pred = clf_DT.predict(X_validation_normalized)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [65]:
from sklearn.svm import LinearSVC

clf_SVM = LinearSVC().fit(X_train_normalized, y_train)
y_pred = clf_SVM.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2488
           1       0.90      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8946


### Only CountVectorizer()

In [66]:
# Bag of Words vectorization
cv = CountVectorizer().fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

In [67]:
clf_NB = MultinomialNB().fit(X_train_counts, y_train)
y_pred = clf_NB.predict(X_validation_counts)

# X_train_normalized = np.array(X_train_normalized)

# print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84      2488
           1       0.83      0.88      0.85      2512

   micro avg       0.85      0.85      0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000

Accuracy % =  0.846


In [68]:
clf_LR = LogisticRegression().fit(X_train_counts, y_train)
y_pred = clf_LR.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2488
           1       0.90      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8896


In [69]:
# clf_DT = DecisionTreeClassifier().fit(X_train_counts, y_train)
# y_pred = clf_DT.predict(X_validation_counts)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [70]:
clf_SVM = LinearSVC().fit(X_train_counts, y_train)
y_pred = clf_SVM.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2488
           1       0.88      0.86      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8686


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### CountVectorizer() with tf-idf

In [ ]:
# Bag of Words vectorization
cv = CountVectorizer().fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

# tfidf
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_validation_tfidf = tfidf_transformer.transform(X_validation_counts)

# normalization
normalizer_tranformer = Normalizer().fit(X=X_train_tfidf)
X_train_normalized = normalizer_tranformer.transform(X_train_tfidf)
X_validation_normalized = normalizer_tranformer.transform(X_validation_tfidf)

In [82]:
clf_NB = MultinomialNB().fit(X_train_normalized, y_train)
y_pred = clf_NB.predict(X_validation_normalized)

# X_train_normalized = np.array(X_train_normalized)

print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

(20000, 68539)
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      2477
           1       0.84      0.88      0.86      2523

   micro avg       0.86      0.86      0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000

Accuracy % =  0.8564


In [83]:
clf_LR = LogisticRegression().fit(X_train_normalized, y_train)
y_pred = clf_LR.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.87      0.90      0.89      2477
           1       0.90      0.87      0.89      2523

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8868


In [75]:
# clf_DT = DecisionTreeClassifier().fit(X_train_normalized, y_train)
# y_pred = clf_DT.predict(X_validation_normalized)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [84]:
clf_SVM = LinearSVC().fit(X_train_normalized, y_train)
y_pred = clf_SVM.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      2477
           1       0.90      0.89      0.89      2523

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8934
